In [ ]:
!pip install evaluate

In [ ]:
!pip install -q evaluate
!pip install -q transformers==4.40.2 --upgrade
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.


In [ ]:
import pandas as pd
import random
from tqdm import tqdm
from transformers import pipeline

# === CONFIGURATION ===
MODEL = "deepset/gbert-base"   # Hugging Face model name
DEVICE = 0  # -1 for CPU, 0 for GPU

# Classes used
CLASSES = [
    "Prämisse",
    "Behauptung",
    "Schlussfolgerung",
    "Nicht-argumentativer Text"
]

# Mapping for evaluation
label2id = {label: i for i, label in enumerate(CLASSES)}
id2label = {i: label for label, i in label2id.items()}

# === LOAD DATA ===
#Original text

df = pd.read_csv(
    "https://raw.githubusercontent.com/happy522/Argument-Mining/refs/heads/main/AM_gelabeled-2.csv"
).copy().sample(frac=1, random_state=42).reset_index(drop=True)


print(df.info())
import pandas as pd

# Suppose your DataFrame is called df

# Classes you want to keep
keep_classes = [
    "Prämisse",
    "Behauptung",
    "Schlussfolgerung",
    "Nicht-argumentativer Text"
]

# Remove leading "/" in case your labels look like "/Zurückweisung"
df["Label"] = df["Label"].str.replace("/", "", regex=False).str.strip()

# Filter DataFrame
filtered_df = df[df["Label"].isin(keep_classes)]

# Reset index if needed
df = filtered_df.reset_index(drop=True)

print(df)

# === LOAD MODEL ===
classifier = pipeline(
    "zero-shot-classification",
    model=MODEL,
    device=DEVICE
)

# === RUN INFERENCE ===
predictions = []
for text in tqdm(df["Komponente"], desc="Classifying"):
    try:
        result = classifier(
            sequences=text,
            candidate_labels=CLASSES,
            hypothesis_template="Dieser Text ist {}."  # German template
        )
        pred = result["labels"][0]  # take top prediction
    except Exception as e:
        print("Error:", e)
        pred = "Nicht-argumentativer Text"
    predictions.append(pred)

df["prediction"] = predictions
print(df[["Komponente", "prediction"]].head())
import pandas as pd
import evaluate
from sklearn.metrics import classification_report
import numpy as np
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])
# === 1. Define your classes ===

# === 2. Clean label column (remove leading slashes) ===
df["Label"] = df["Label"].str.replace("/", "", regex=False).str.strip()
df["prediction"] = df["prediction"].str.strip()  # also strip prediction

# === 3. Map labels to integers ===
label2id = {label: idx for idx, label in enumerate(CLASSES)}
id2label = {idx: label for idx, label in enumerate(CLASSES)}

y_true = df["Label"].map(label2id)
y_pred = df["prediction"].map(label2id)

# Check unmapped labels
print("Unmapped true labels:", df["Label"][y_true.isna()].unique())
print("Unmapped predicted labels:", df["prediction"][y_pred.isna()].unique())

# === 4. Compute accuracy and macro-F1 using  evaluate ===
metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

accuracy = metric_acc.compute(predictions=y_pred, references=y_true)
f1 = metric_f1.compute(predictions=y_pred, references=y_true, average="macro")

print("\n=== RESULTS ===")
print("Accuracy:", accuracy["accuracy"])
print("Macro-F1:", f1["f1"])

# === 5. Classification report with all classes ===
print("\n=== Classification Report ===\n")
print(classification_report(
    y_true,
    y_pred,
    labels=range(len(CLASSES)),
    target_names=CLASSES,
    digits=4,
    zero_division=0  # avoids errors for missing classes
))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1272 entries, 0 to 1271
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1272 non-null   int64 
 1   Komponente  1272 non-null   object
 2   Label       1272 non-null   object
dtypes: int64(1), object(2)
memory usage: 29.9+ KB
None
      Unnamed: 0                                         Komponente  \
0            208  Deswegen versucht die AfD unsere Demos mit all...   
1            966  Die Deportation und Ermordung von südwestdeuts...   
2            714  Am 27.01. ist der Gedenktag an die Opfer des N...   
3            584  Die Stadt Potsdam hat ein Einreiseverbot gegen...   
4             63  "Damit eine Protestbewegung Erfolg hat, muss n...   
...          ...                                                ...   
1068         330  Spätestens jetzt, nach den Enthüllungen über d...   
1069         466  Bernd.. Björn.. Adolf.. egal.. Dem Nazi und se...   

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Classifying: 100%|██████████| 1073/1073 [00:57<00:00, 18.67it/s]


                                          Komponente  \
0  Deswegen versucht die AfD unsere Demos mit all...   
1  Die Deportation und Ermordung von südwestdeuts...   
2  Am 27.01. ist der Gedenktag an die Opfer des N...   
3  Die Stadt Potsdam hat ein Einreiseverbot gegen...   
4  "Damit eine Protestbewegung Erfolg hat, muss n...   

                  prediction  
0                   Prämisse  
1                 Behauptung  
2  Nicht-argumentativer Text  
3           Schlussfolgerung  
4                   Prämisse  
Unmapped true labels: []
Unmapped predicted labels: []

=== RESULTS ===
Accuracy: 0.3681267474370923
Macro-F1: 0.2629010032726626

=== Classification Report ===

                           precision    recall  f1-score   support

                 Prämisse     0.2284    0.1917    0.2085       193
               Behauptung     0.2500    0.1863    0.2135       204
         Schlussfolgerung     0.0299    0.5556    0.0567        18
Nicht-argumentativer Text     0.7311    0.4711

In [ ]:
df


,Komponente,Label,prediction
0,Deswegen versucht die AfD unsere Demos mit all...,Schlussfolgerung,Prämisse
1,Die Deportation und Ermordung von südwestdeuts...,Nicht-argumentativer Text,Behauptung
2,Am 27.01. ist der Gedenktag an die Opfer des N...,Nicht-argumentativer Text,Nicht-argumentativer Text
3,Die Stadt Potsdam hat ein Einreiseverbot gegen...,Nicht-argumentativer Text,Schlussfolgerung
4,"""Damit eine Protestbewegung Erfolg hat, muss n...",Prämisse,Prämisse
...,...,...,...
1068,"Spätestens jetzt, nach den Enthüllungen über d...",Behauptung,Behauptung
1069,Bernd.. Björn.. Adolf.. egal.. Dem Nazi und se...,Nicht-argumentativer Text,Schlussfolgerung
1070,Diskriminierung und auf Demokratiefeindlichkei...,Prämisse,Schlussfolgerung
1071,\tEin geiles lied von Heiter bis Wolkig wo lei...,Nicht-argumentativer Text,Prämisse


In [ ]:
df.to_csv("Zero_shot.csv", index=False, encoding="utf-8")

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import evaluate
from sklearn.metrics import classification_report
import numpy as np

# === CONFIG ===
DEVICE = 0   # use 0 for first GPU, or -1 for CPU
CLASSES = [
    "Prämisse",
    "Behauptung",
    "Schlussfolgerung",
    "Nicht-argumentativer Text"
]

# Models to compare (the five recommended above)
MODELS = [
    "deepset/gbert-base",
    "joeddav/xlm-roberta-large-xnli",
    "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7",
    "svalabs/gbert-large-zeroshot-nli",
    "vicgalle/xlm-roberta-large-xnli-anli",
    "61347023S/xlm-roberta-large-xnli-v2.0"
]

# === Load data (same URL you used) ===
df = pd.read_csv(
    "https://raw.githubusercontent.com/happy522/Argument-Mining/refs/heads/main/AM_gelabeled-2.csv"
).sample(frac=1, random_state=42).reset_index(drop=True)

# Clean label column
if "Label" not in df.columns:
    raise RuntimeError("Expected column 'Label' not found in CSV.")
df["Label"] = df["Label"].astype(str).str.replace("/", "", regex=False).str.strip()
df = df[df["Label"].isin(CLASSES)].reset_index(drop=True)

# Map labels to ints
label2id = {label: i for i, label in enumerate(CLASSES)}
id2label = {i: label for label, i in label2id.items()}

# Evaluation helpers
metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

summary_rows = []

for model_name in MODELS:
    print("\n" + "="*80)
    print(f"MODEL: {model_name}")
    print("="*80)
    try:
        classifier = pipeline(
            "zero-shot-classification",
            model=model_name,
            device=DEVICE
        )
    except Exception as e:
        print(f"Failed to load pipeline for {model_name}: {e}")
        summary_rows.append({
            "model": model_name,
            "error": str(e)
        })
        continue

    preds = []
    # Run inference (batching per example)
    for text in tqdm(df["Komponente"].astype(str), desc=f"Classifying with {model_name}"):
        try:
            out = classifier(
                sequences=text,
                candidate_labels=CLASSES,
                hypothesis_template="Dieser Text ist {}."
            )
            # top label
            pred_label = out["labels"][0]
        except Exception as e:
            # fallback: assign the non-argumentative label (or choose randomly)
            print("Inference error for one example:", e)
            pred_label = "Nicht-argumentativer Text"
        preds.append(pred_label.strip())

    df_model = df.copy()
    df_model["prediction"] = preds

    # Map to ints; handle any unmapped label by treating it as 'Nicht-argumentativer Text' index
    y_true = df_model["Label"].map(label2id)
    y_pred = df_model["prediction"].map(label2id)

    # If any predictions didn't map, force them to the index of 'Nicht-argumentativer Text'
    missing_pred_mask = y_pred.isna()
    if missing_pred_mask.any():
        print(f"Warning: {missing_pred_mask.sum()} unmapped predictions found for {model_name}. Mapping them to 'Nicht-argumentativer Text'.")
        y_pred = y_pred.fillna(label2id["Nicht-argumentativer Text"])

    # Compute metrics
    acc_res = metric_acc.compute(predictions=y_pred.astype(int).tolist(), references=y_true.astype(int).tolist())
    f1_res = metric_f1.compute(predictions=y_pred.astype(int).tolist(), references=y_true.astype(int).tolist(), average="macro")

    print(f"\nResults for {model_name}:")
    print("Accuracy:", acc_res["accuracy"])
    print("Macro-F1:", f1_res["f1"])

    print("\nClassification report:")
    print(classification_report(
        y_true.astype(int),
        y_pred.astype(int),
        labels=list(range(len(CLASSES))),
        target_names=CLASSES,
        digits=4,
        zero_division=0
    ))

    summary_rows.append({
        "model": model_name,
        "accuracy": float(acc_res["accuracy"]),
        "macro_f1": float(f1_res["f1"])
    })

# Summary
summary_df = pd.DataFrame(summary_rows)
print("\n=== SUMMARY ===")
print(summary_df)

# Optional: save results
summary_df.to_csv("model_comparison_summary.csv", index=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



MODEL: deepset/gbert-base


Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Classifying with deepset/gbert-base: 100%|██████████| 1073/1073 [01:04<00:00, 16.69it/s]



Results for deepset/gbert-base:
Accuracy: 0.18359739049394222
Macro-F1: 0.1562926990879257

Classification report:
                           precision    recall  f1-score   support

                 Prämisse     0.1818    0.0311    0.0531       193
               Behauptung     0.2698    0.4853    0.3468       204
         Schlussfolgerung     0.0060    0.1667    0.0116        18
Nicht-argumentativer Text     0.5086    0.1353    0.2137       658

                 accuracy                         0.1836      1073
                macro avg     0.2415    0.2046    0.1563      1073
             weighted avg     0.3960    0.1836    0.2067      1073


MODEL: joeddav/xlm-roberta-large-xnli


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Classifying with joeddav/xlm-roberta-large-xnli: 100%|██████████| 1073/1073 [02:18<00:00,  7.75it/s]


...


Results for svalabs/gbert-large-zeroshot-nli:
Accuracy: 0.18732525629077354
Macro-F1: 0.1542317475847067

Classification report:
                           precision    recall  f1-score   support

                 Prämisse     0.2150    0.3575    0.2685       193
               Behauptung     0.1761    0.6275    0.2750       204
         Schlussfolgerung     0.0769    0.0556    0.0645        18
Nicht-argumentativer Text     0.2500    0.0046    0.0090       658

                 accuracy                         0.1873      1073
                macro avg     0.1795    0.2613    0.1542      1073
             weighted avg     0.2267    0.1873    0.1071      1073


MODEL: vicgalle/xlm-roberta-large-xnli-anli


Device set to use cuda:0
Classifying with vicgalle/xlm-roberta-large-xnli-anli: 100%|██████████| 1073/1073 [02:17<00:00,  7.82it/s]



Results for vicgalle/xlm-roberta-large-xnli-anli:
Accuracy: 0.19944082013047532
Macro-F1: 0.15358720967597772

Classification report:
                           precision    recall  f1-score   support

                 Prämisse     0.1818    0.0829    0.1139       193
               Behauptung     0.2136    0.7255    0.3300       204
         Schlussfolgerung     0.0253    0.3333    0.0471        18
Nicht-argumentativer Text     0.8000    0.0669    0.1234       658

                 accuracy                         0.1994      1073
                macro avg     0.3052    0.3021    0.1536      1073
             weighted avg     0.5643    0.1994    0.1597      1073


MODEL: 61347023S/xlm-roberta-large-xnli-v2.0


Device set to use cuda:0
Classifying with 61347023S/xlm-roberta-large-xnli-v2.0: 100%|██████████| 1073/1073 [01:11<00:00, 14.92it/s]



Results for 61347023S/xlm-roberta-large-xnli-v2.0:
Accuracy: 0.34575955265610436
Macro-F1: 0.24559380439509615

Classification report:
                           precision    recall  f1-score   support

                 Prämisse     0.1754    0.0518    0.0800       193
               Behauptung     0.2404    0.7059    0.3587       204
         Schlussfolgerung     0.0354    0.3889    0.0648        18
Nicht-argumentativer Text     0.9589    0.3191    0.4789       658

                 accuracy                         0.3458      1073
                macro avg     0.3525    0.3664    0.2456      1073
             weighted avg     0.6659    0.3458    0.3773      1073


=== SUMMARY ===
                                               model  accuracy  macro_f1
0                                 deepset/gbert-base  0.183597  0.156293
1                     joeddav/xlm-roberta-large-xnli  0.198509  0.140675
2  MoritzLaurer/mDeBERTa-v3-base-xnli-multilingua...  0.189189  0.163679
3               